## Extraccion con APIs

In [ ]:
# Tratamiento de datos
import pandas as pd

# Uso de API's
import requests

# Importar librerías para procesamiento de texto
import re

# Configuración
pd.set_option('display.max_columns', None)

In [3]:
def peliculas_uno(genero, anyo):

	url = "https://moviesdatabase.p.rapidapi.com/titles"
 #diccionario donde se guarda cada pagina
	paginas = {}
	
	#recorro de la pagina 1 a la 10
	for p in range (1,10):
     
		querystring =  {"genre":{genero},"year":{anyo},"page":{p},"limit":"50"}


		headers = {
			"X-RapidAPI-Key": "a2850b5a12msh0b9b93d5cd14a80p1d8e67jsnefeba741d833",
			"X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
		}

		response = requests.get(url, headers=headers, params=querystring)
	#lo guardo dentro de un diccionario nuevo dentro del diccionario
		paginas[p] = response.json()
		 
 #devuelvo el diccionario con las paginas dentro
	return paginas


In [4]:
#codigo franca
def tuplas_en_lista(respuestas):

    lista_tuplas = []

    for p in respuestas.values():
        #genero 
        cadena = p["next"]
        genero = cadena.split('=')[-1]
        #iterar en results
        for result in p["results"]:
            peliculas_drama = []
            if result["titleType"]["text"] != "Short" and result["titleType"]["text"] != "Movie":
              pass
            else:
                #id
                peliculas_drama.append(result["id"])
                #nombre
                peliculas_drama.append(result["titleText"]["text"])
                #año
                peliculas_drama.append(result["releaseDate"]["year"])
                #mes
                peliculas_drama.append(result["releaseDate"]["month"])
                #genero
                peliculas_drama.append(genero)
   
                lista_tuplas.append(tuple(peliculas_drama))
    return lista_tuplas

In [5]:
#codigo Valen
def tuplas_en_lista2(respuestas):

    lista_tuplas = []

    for p in respuestas.values():
        #genero 
        cadena = p["next"]
        genero = cadena.split('=')[-1]
        #iterar en results
        for result in p["results"]:
            peliculas_drama = []
            if result["titleType"]["text"] != "Short" and result["titleType"]["text"] != "Movie":
              pass
            else:
                if result["releaseDate"] is not None:
                    peliculas_drama.append(result["id"])
                    #nombre
                    peliculas_drama.append(result["titleText"]["text"])
                    #año
                    peliculas_drama.append(result["releaseDate"]["year"])
                    #mes
                    peliculas_drama.append(result["releaseDate"]["month"])
                    #genero
                    peliculas_drama.append(genero)
                # Continuar con el procesamiento
                else:
                    continue
   
                lista_tuplas.append(tuple(peliculas_drama))
    return lista_tuplas

## Extraccion con Selenium desde https://www.imdb.com/
Una vez que tengan una lista de películas de la API, el siguiente paso es obtener información más detallada sobre ellas. Utilizaras la herramienta de automatización web Selenium para navegar a sitios web de reseñas de películas (IMDB) y extraer detalles como calificaciones, actores y directores. En concreto deberás extraer información sobre:

1. Puntuacion de IMDB (en caso de que la tenga).

2. Dirección (director/a o directore/as de cada película).

3. Gionistas (de cada película).

4. Argumento.

5. Duración (en minutos).

6. Nombre de la película

In [ ]:
!pip install selenium

In [3]:
# Importar librerías para tratamiento de datos
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
from bs4 import BeautifulSoup
import requests

# Importar librerías para automatización de navegadores web con Selenium
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# Importar librerías para pausar la ejecución
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.
from selenium.common.exceptions import NoSuchElementException #para manejarr errores
from selenium.common.exceptions import JavascriptException
# Configuraciones
pd.set_option('display.max_columns', None) 
import re

In [4]:
# Resultado de la fase 1
lista_movies_ejemplo = ["tt0120338"]

In [6]:
# Funcion para sacar datos de los actores de las peliculas en IMBD a partir de una lista de tuplas de peliculas
def actores_imdb(lista_movies_ejemplo):
    
    # Inicializa el navegador Chrome
    driver = webdriver.Chrome()
    # Maximiza la ventana
    driver.maximize_window()
    sleep(3)

    # Navega a la página web "https://www.imdb.com/"
    driver.get("https://www.imdb.com/")

    # Espera durante 3 segundos (para permitir que la página cargue completamente)
    sleep(3)
    # Acepta las cookies haciendo clic en el elemento con el XPath
    driver.find_element("xpath", "/html/body/div[2]/div/div/div[2]/div/button[2]").click()
    sleep(2)

    # Crear lista donde ir guardando las tuplas de cada pelicula
    actores_lista = [] #contiene nombre, anyo y luego otra lista dentro de la lista con los otros 3 elementos


    #Itero por la lista de peliculas por id
    for id in lista_movies_ejemplo:
    # Introducir id de pelicula en el campo de búsqueda y presionar Enter
        driver.find_element("css selector", "#suggestion-search").send_keys(id, Keys.ENTER)
        sleep (3)

        while len(actores_lista) < 10:
            # Localizar el elemento que deseas hacer visible
            i = 1
            while True:
                try:
                    actores = driver.find_element(By.XPATH, f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/div[2]/div[2]/div[{i}]/div[2]/a')
                    # Desplazar la página para hacer visible el elemento
                    driver.execute_script("arguments[0].scrollIntoView(true);", actores)
                    # Hacer clic en el elemento
                    actores.click()
                    sleep (3)
                    

                    #Extraemos 'Nombre'
                    try:
                        nombre = driver.find_element ("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/h1/span').text
                        actores_lista.append (nombre)
                    except:
                        try:
                            nombre = driver.find_element ("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-4e4cc5f9-3.dDRspk > div > h1 > span').text
                            actores_lista.append (nombre)
                        except (NoSuchElementException, JavascriptException):
                            actores_lista.append ('No info')
                            pass
                
                    #Extraemos anyo de nacimiento
                    try:
                        fecha_nac = driver.find_element ("xpath", """//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[6]/div[2]/ul/li[4]/div/ul/li[1]/a""").text
                        patron = r'\b\d{4}\b'
                        anyo = re.findall(patron, fecha_nac)
                        actores_lista.append (anyo)

                    except:
                        try:
                            fecha_nac = driver.find_element ("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(20) > div.sc-f65f65be-0.bBlII > ul > li:nth-child(2) > div > ul > li.ipc-inline-list__item.test-class-react > a").text
                            patron = r'\b\d{4}\b'
                            anyo = re.findall(patron, fecha_nac)
                            actores_lista.append (anyo)  

                        except (NoSuchElementException, JavascriptException):
                            actores_lista.append ('No info')
                            pass
                    sleep(2)
                    
                    #Extraemos 'Conocido por'
                    lista_conocido = []
                    for c in range (1,4):
                        try: 
                            conocido = driver.find_element("xpath", f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/div[3]/section[1]/div[2]/div/div[2]/div[{c}]/div[2]/div[1]/a').text
                            lista_conocido.append (conocido)
                        except (NoSuchElementException, JavascriptException):
                            actores_lista.append ('No info')
                            pass
                    actores_lista.append(lista_conocido)
                    
                    sleep (2)
                    
                    #Extraemos que hace?
                    lista_que_hace = []
                    for q in range(4, 11, 2):
                        try:
                            que_hace = driver.find_element("xpath", f"""//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/div[3]/section[2]/div[{q}]/hgroup/h3""").text
                            lista_que_hace.append(que_hace)
                            sleep(2)
                            
                        except (NoSuchElementException, JavascriptException):
                            actores_lista.append ('No info')
                            pass
                    actores_lista.append(lista_que_hace)  

                    sleep (2)
                    #Extraemos si tiene premios oscar
                    lista_premios = []
                    try:
                        premio_oscar = driver.find_element("xpath", """//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[1]/div/ul/li/a[1]""").text
                        lista_premios.append(premio_oscar)
                    except:
                        try:
                            premio_oscar = driver.find_element("css selector", """//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[1]/div/ul/li/a[1]""").text
                            lista_premios.append(premio_oscar)

                        except (NoSuchElementException, JavascriptException):
                            actores_lista.append ('No info')
                            pass   
                    
                    #Extraemos otros premios:
                    try:
                        driver.find_element("xpath", """//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[1]/div/ul/li/a[1]""").click()
                        sleep (3)
                    
                    except:
                        try:
                            driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3) > div > ul > li > div > ul > li > span").click()
                        except (NoSuchElementException, JavascriptException):
                            actores_lista.append ('No info')
                            pass

                    for p in range (1,4):

                        try:
                            otros_premios = driver.find_element("css selector", f'#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child({p}) > div.sc-f65f65be-0.bBlII > ul > li > div.ipc-metadata-list-summary-item__c > div > a').text
                            lista_premios.append(otros_premios)
                        except (NoSuchElementException, JavascriptException):
                            try:
                                otros_premios = driver.find_element("xpath", f'//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[{p}]/div[2]/ul/li/div[2]/div/a').text
                                lista_premios.append(otros_premios)
                            
                            except (NoSuchElementException, JavascriptException):
                                actores_lista.append ('No info')
                                pass
                    actores_lista.append(lista_premios)                
                            
                    i += 1            
                except (NoSuchElementException, JavascriptException):
                    actores_lista.append("Desconocido")
                    pass
                
                driver.quit()
                sleep (3)
        return actores_lista

In [ ]:
actores_imdb(lista_movies_ejemplo)